# Notebook 1 - preprocessing

## Intro

This Jupyter notebook presents part of the model behind the publication: 
> Krych, K. & Pettersen, JB. (2024). Long-term lifetime trends of large appliances since the introduction in Norwegian households. Journal of Industrial Ecology. 

Here, we preprocess the data, e.g., derive complete time series based on data points through interpolation and regression.

## Imports and parameter definition

In [42]:
import os
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import scipy.stats
from scipy.optimize import curve_fit

In [2]:
durables = ['fridge & fridge freezer' ,'freezer','washing machine', 'tumble dryer', 'dishwasher', 'oven']
TimeStart = 1940
TimeEnd = 2022
MyYears = list(range(TimeStart,TimeEnd+1))
excel = os.path.abspath(os.path.join(os.getcwd(), 'data.xlsx'))

In [19]:
def df_to_excel_overlay(excel, df, sheet_name):
    writer = pd.ExcelWriter(excel, mode='a', if_sheet_exists="overlay", engine='openpyxl') 
    df.to_excel(writer, sheet_name=sheet_name, index=False,startrow=1, header=False)
    writer.close()

## Inflows of appliances (I) - interpolation

In [3]:
df_i_data = pd.read_excel(excel, sheet_name='I_data')

In [4]:
def perform_inflows_interpolation(df_d, durable, MyYears):
    min_year = min(df_d['time']) # the first year with data
    max_year = max(df_d['time']) # the last year with data
    if min_year > MyYears[0]:
        df_d = pd.concat([df_d, pd.DataFrame({"time": [MyYears[0]], "durable": [durable], "value": [0]})], ignore_index=True)
        df_d = pd.concat([df_d, pd.DataFrame({"time": [min_year-1], "durable": [durable], "value": [0]})], ignore_index=True)
    if max_year < MyYears[-1]:
        last_value = df_d[df_d['time']==max_year]['value'].item()
        df_d = pd.concat([df_d, pd.DataFrame({"time": [MyYears[-1]], "durable": [durable], "value": [last_value]})], ignore_index=True)
    df_d = df_d.groupby('time').mean().reset_index()
    x_data = df_d['time']
    y_data = df_d['value']
    f_linear = interp1d(x_data, y_data)
    return f_linear(MyYears)

In [5]:
df_i_ip = pd.DataFrame(index=pd.MultiIndex.from_product([durables, MyYears], names=['durable','time']), columns=['value'])
for durable in durables:
    df_d = df_i_data[df_i_data['durable'] == durable]
    df_i_ip.loc[durable,:] = perform_inflows_interpolation(df_d, durable, MyYears)
df_i_ip = df_i_ip.reset_index()

,durable,time,value
0,fridge & fridge freezer,1940,0.0
1,fridge & fridge freezer,1941,0.0
2,fridge & fridge freezer,1942,0.0
3,fridge & fridge freezer,1943,0.0
4,fridge & fridge freezer,1944,0.0
...,...,...,...
493,oven,2018,208164.5
494,oven,2019,207401.5
495,oven,2020,214029.5
496,oven,2021,215359.0


In [ ]:
df_to_excel_overlay(excel,df_i_ip,'I')

## People per dwelling (PpD) - regression

In [9]:
df_ppd_data = pd.read_excel(excel, sheet_name='PpD_data')

In [10]:
def logistic(x, ti, a, C0, C1):
    """
    ti: inflection time
    a: slope
    C0: start value
    C1: end value
    x: vector of observation points (time)
    """
    return (C1 - C0) / (1 + np.exp(-a * (x - ti))) + C0

In [21]:
lower_bounds = [1900, 0, 5, 0]
higher_bounds = [2100, 1, 10, 10]
popt, pcov = curve_fit(logistic, df_ppd_data['time'], df_ppd_data['value'], bounds=[lower_bounds, higher_bounds])
df_ppd_rg = pd.DataFrame(data=logistic(MyYears,*popt), index=pd.MultiIndex.from_product([MyYears], names=['time']), columns=['value'])
df_ppd_rg = df_ppd_rg.reset_index()
df_ppd_rg

,time,value
0,1940,3.826035
1,1941,3.791589
2,1942,3.757082
3,1943,3.722542
4,1944,3.687997
...,...,...
78,2018,2.172932
79,2019,2.167046
80,2020,2.161374
81,2021,2.155910


In [22]:
df_to_excel_overlay(excel,df_ppd_rg,'PpD')

## Inflows of cabins (I-cab) - interpolation

In [26]:
df_i_cab_data = pd.read_excel(excel, sheet_name='I-cab_data', usecols="A:D")
f_cabins = interp1d(list(df_i_cab_data['time']), list(df_i_cab_data['value']))
MyYears_extended = list(range(1900,TimeEnd+1))
df_i_cab_ip = pd.DataFrame(data=f_cabins(MyYears_extended), index=pd.MultiIndex.from_product([MyYears_extended], names=['time']), columns=['value'])
df_i_cab_ip = df_i_cab_ip.reset_index()
df_i_cab_ip

,time,value
0,1900,0.000000
1,1901,833.333333
2,1902,1666.666667
3,1903,2500.000000
4,1904,3333.333333
...,...,...
118,2018,431028.000000
119,2019,434809.000000
120,2020,437833.000000
121,2021,440443.000000


In [28]:
df_to_excel_overlay(excel,df_i_cab_ip,'I-cab')

## Share of cabins electrified (SoCE) - regression

In [37]:
df_soce = pd.read_excel(excel, sheet_name='SoCE_data', usecols="A:D")
def linear(x, a, b):
    return a * x + b
x_data, y_data = df_soce['time'], df_soce['value']
with np.errstate(divide='ignore'): # to ignore "RuntimeWarning: divide by zero encountered in..."
    y_data_log = np.log(y_data*100)
y_data_log[y_data_log == -np.inf] = 0
popt, pcov = curve_fit(linear, x_data, y_data_log)
soce = np.exp(popt[0]*np.array(MyYears)+popt[1])/100
soce[soce >1] = 1
soce[:1960-TimeStart] = 0 
soce[soce <0] = 0
df_soce_rg = pd.DataFrame(data=soce, index=pd.MultiIndex.from_product([MyYears], names=['time']), columns=['value'])
df_soce_rg = df_soce_rg.reset_index()
df_soce_rg

,time,value
0,1940,0.0
1,1941,0.0
2,1942,0.0
3,1943,0.0
4,1944,0.0
...,...,...
78,2018,1.0
79,2019,1.0
80,2020,1.0
81,2021,1.0


In [38]:
df_to_excel_overlay(excel,df_soce_rg,'SoCE')

## The number of dwellings

The number of dwellings is calculated using various model parameters. During the uncertainty analysis, the values of these parameters might vary, which is why in the preliminary analysis, the number of dwellings is calculated for each model run. However, in the simplified analysis, where many of the parameters are considered fixed, the number of dwellings is also a fixed parameter. Pre-calculating the number of dwellings improves the computational time in the final analysis.

In [43]:
df_ppd = pd.read_excel(excel, sheet_name='PpD')
df_p = pd.read_excel(excel, sheet_name='P')
df_c = pd.read_excel(excel, sheet_name='I-cab')
df_soce = pd.read_excel(excel, sheet_name='SoCE')
df_k_cab = pd.read_excel(excel, sheet_name='k-cab')
df_l_cab = pd.read_excel(excel, sheet_name='lambda-cab')

In [44]:
dwellings = np.array(df_p['value']/df_ppd['value'])

t = df_c['time']
s = df_c['value']
scale = df_l_cab['value'].values[0]
shape = df_k_cab['value'].values[0]
sf = np.zeros((len(t), len(t)))
for m in range(0, len(t)):  # cohort index
    sf[m::,m] = scipy.stats.weibull_min.sf(np.arange(0,len(t)-m), c=shape, loc = 0, scale=scale)

# MFA calculations start (assuming sf[0] != 0 and no negative inflows)
i = np.zeros(len(t))
s_c = np.zeros((len(t), len(t)))
i[0] = s[0] / sf[0, 0]
s_c[:, 0] = i[0] * sf[:, 0]
for m in range(1, len(t)):
    i[m] = (s[m] - s_c[m, :].sum()) / sf[m,m]
    s_c[m::, m] = i[m] * sf[m::, m]

o_c = np.zeros_like(s_c)
o_c[1::,:] = -1 * np.diff(s_c,n=1,axis=0)
o_c[np.diag_indices(len(t))] = i - np.diag(s_c) # allow for outflow in year 0 already

soce = df_soce['value'] # share of cabins electrified
soce[soce >1] = 1
soce[:1960-TimeStart] = 0 
soce[soce <0] = 0
el_cabins = np.einsum('tc,c->t',s_c[40:,40:],soce)
all_dwellings = dwellings+el_cabins

C:\Users\kamilami\AppData\Local\Temp/ipykernel_9444/1887577306.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soce[soce >1] = 1
C:\Users\kamilami\AppData\Local\Temp/ipykernel_9444/1887577306.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soce[:1960-TimeStart] = 0
C:\Users\kamilami\AppData\Local\Temp/ipykernel_9444/1887577306.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soce[soce <0] = 0


In [50]:
df_d = pd.DataFrame(data=all_dwellings, index=pd.MultiIndex.from_product([MyYears], names=['time']), columns=['value'])
df_d = df_d.reset_index()
df_d['unit'] = 'dwellings'
df_d['source'] = 'calculated using a dwelling sub-model'
df_to_excel_overlay(excel,df_d,'D')